# Phase 4: Continuous Learning
- Enable continuous learning orchestrator (memory-only or with processor).
- Auto-ingest `vocab_src` feeds (JSONL/CSV) into hippocampal memory.

In [ ]:
%pip install -q torch datasets transformers sentencepiece tqdm
import torch, sys, asyncio
from pathlib import Path
ROOT = Path('/content/repo') if Path('/content/repo').exists() else Path.cwd().parent
sys.path.insert(0, str(ROOT))
from colab_l4_training import get_test_config, main, ingest_jsonl_to_memory, ingest_csv_pairs_to_memory

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

In [ ]:
# Load model with continuous learning enabled (memory-only orchestrator)
config = get_test_config()
config.enable_continuous_learning = True
model, losses, cl_orch = main(config_preset='test')
hippocampus = model.hippocampus
print('Orchestrator available:', cl_orch is not None)
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-base', legacy=True)

In [ ]:
# Manual ingestion of vocab_src corpora (optional, besides orchestrator)
jsonl_paths = [
    ROOT/'vocab_src/instruct_55k_clean.jsonl',
    ROOT/'vocab_src/OpenThoughts-114k.jsonl',
    ROOT/'vocab_src/principles_texts.jsonl',
]
for p in jsonl_paths:
    if p.exists():
        stored = ingest_jsonl_to_memory(str(p), tokenizer, model, hippocampus, device, max_items=500)
        print(p.name, 'stored', stored)

csv_path = ROOT/'vocab_src/timeline_conversations.csv'
if csv_path.exists():
    stored = ingest_csv_pairs_to_memory(str(csv_path), tokenizer, model, hippocampus, device, max_items=500)
    print(csv_path.name, 'stored', stored)

print('Total memories:', hippocampus.memory_count)

In [ ]:
# Start/stop the orchestrator (memory-only) to continuously ingest feeds
async def run_orchestrator(duration_sec=120):
    if cl_orch is None:
        print('No orchestrator built (enable_continuous_learning must be True)')
        return
    await cl_orch.start()
    await asyncio.sleep(duration_sec)
    await cl_orch.stop()

# To run: await run_orchestrator(120)
# Note: In notebooks, run this in an async cell or use asyncio.run in a fresh kernel.